In [ ]:
pip install recipe-scrapers

In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from recipe_scrapers import scrape_me
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [5]:
linklist=[]
for i in range(1,5):
    responses=requests.get('https://www.allrecipes.com/recipes/80/main-dish/?page={}'.format(i))
    soup=BeautifulSoup(responses.content,'html.parser')
    for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
        linklist.append(link.get('href'))

# Filter through links that correspond with a singular dish and not multiple dishes       
dishes = [k for k in linklist if '/recipe/' in k]

# Remove duplicate links
recipes = [] 
for d in dishes: 
    if d not in recipes: 
        recipes.append(d)


In [6]:
len(recipes)

97

In [7]:
# Use Recipe Scraper package on links
data = []
for links in recipes:
    data.append(scrape_me(links))

In [8]:
# Create lists for name and ingredients of recipes
names = []
ings = []
for dat in data:
    names.append(dat.title())
    ings.append(dat.ingredients())

In [19]:
ings1 = ings
ings1[1]

['2 pounds boneless pork chops',
 '1/2 teaspoon paprika',
 '1 pinch kosher salt and ground black pepper to taste',
 '1/4 cup butter, divided',
 '1 (8 ounce) package sliced fresh mushrooms',
 '4 cloves garlic, minced',
 '1 teaspoon Dijon mustard',
 '2 tablespoons all-purpose flour',
 '2 cups beef broth']

In [11]:
import re


SEPARATOR_RE = re.compile(r'^([\d\s*[\d\.,/]*)\s*(.+)')


def normalize(st):
    """

    :param st:
    :return:
    """
    return re.sub(r'\s+', ' ', SEPARATOR_RE.sub('\g<1> \g<2>', st)).strip()


def escape_re_string(text):
    """

    :param text:
    :return:
    """
    text = text.replace('.', '\.')
    return re.sub(r'\s+', ' ', text)

In [131]:
from itertools import chain
from re import search

UNITS = {"cup": ["cups", "cup", "c.", "c"], "fluid_ounce": ["fl. oz.", "fl oz", "fluid ounce", "fluid ounces"],
         "gallon": ["gal", "gal.", "gallon", "gallons"], "ounce": ["oz", "oz.", "ounce", "ounces"],
         "pint": ["pt", "pt.", "pint", "pints"], "pound": ["lb", "lb.", "pound", "pounds"],
         "quart": ["qt", "qt.", "qts", "qts.", "quart", "quarts"],
         "tablespoon": ["tbsp.", "tbsp", "T", "T.", "tablespoon", "tablespoons", "tbs.", "tbs"],
         "teaspoon": ["tsp.", "tsp", "t", "t.", "teaspoon", "teaspoons"],
         "gram": ["g", "g.", "gr", "gr.", "gram", "grams"], "kilogram": ["kg", "kg.", "kilogram", "kilograms"],
         "liter": ["l", "l.", "liter", "liters"], "milligram": ["mg", "mg.", "milligram", "milligrams"],
         "milliliter": ["ml", "ml.", "milliliter", "milliliters"], "pinch": ["pinch", "pinches"],
         "dash": ["dash", "dashes"], "touch": ["touch", "touches"], "handful": ["handful", "handfuls"],
         "stick": ["stick", "sticks"], "clove": ["cloves", "clove"], "can": ["cans", "can"], "large": ["large"],
         "small": ["small"], "scoop": ["scoop", "scoops"], "filets": ["filet", "filets"], "sprig": ["sprigs", "sprig"]}

NUMBERS = ['seventeen', 'eighteen', 'thirteen', 'nineteen', 'fourteen', 'sixteen', 'fifteen', 'seventy', 'twelve',
           'eleven', 'eighty', 'thirty', 'ninety', 'twenty', 'seven', 'fifty', 'sixty', 'forty', 'three', 'eight',
           'four', 'zero', 'five', 'nine', 'ten', 'one', 'six', 'two', 'an', 'a', '½', '⅓','¼', '⅛', '¾']

prepositions = ["of"]

a = list(chain.from_iterable(UNITS.values()))
a.sort(key=lambda x: len(x), reverse=True)
a = map(escape_re_string, a)

PARSER_RE = re.compile(
    r'(?P<quantity>(?:[\d\.,][\d\.,\s/]*)?\s*(?:(?:%s)\s*)*)?(\s*(?P<unit>%s)\s+)?(\s*(?:%s)\s+)?(\s*(?P<name>.+))?' % (
        '|'.join(NUMBERS), '|'.join(a), '|'.join(prepositions)))


def parse(st):
    """

    :param st:
    :return:
    """
    st = normalize(st)
    res = PARSER_RE.match(st)
    
    return(res.group('name') or '').strip()

#     return ((res.group('quantity') or '').strip() + ' ' + (res.group('unit') or '').strip(),
#            (res.group('name') or '').strip())



# {
#         'measure': (res.group('quantity') or '').strip() + ' ' + (res.group('unit') or '').strip(),
#         'name': (res.group('name') or '').strip()
#     }


In [75]:
ings

[['1 pound sweet Italian sausage',
  '3/4 pound lean ground beef',
  '1/2 cup minced onion',
  '2 cloves garlic, crushed',
  '1 (28 ounce) can crushed tomatoes',
  '2 (6 ounce) cans tomato paste',
  '2 (6.5 ounce) cans canned tomato sauce',
  '1/2 cup water',
  '2 tablespoons white sugar',
  '1 1/2 teaspoons dried basil leaves',
  '1/2 teaspoon fennel seeds',
  '1 teaspoon Italian seasoning',
  '1 1/2 teaspoons salt, divided, or to taste',
  '1/4 teaspoon ground black pepper',
  '4 tablespoons chopped fresh parsley',
  '12 lasagna noodles',
  '16 ounces ricotta cheese',
  '1 egg',
  '3/4 pound mozzarella cheese, sliced',
  '3/4 cup grated Parmesan cheese'],
 ['2 pounds boneless pork chops',
  '1/2 teaspoon paprika',
  '1 pinch kosher salt and ground black pepper to taste',
  '1/4 cup butter, divided',
  '1 (8 ounce) package sliced fresh mushrooms',
  '4 cloves garlic, minced',
  '1 teaspoon Dijon mustard',
  '2 tablespoons all-purpose flour',
  '2 cups beef broth'],
 ['1 1/2 pounds cub

In [86]:
type(ings2)

list

In [132]:
new_list = []
for sub_list in ings:
    new_list.append([])
    for item in sub_list:
            new_list[-1].append(parse(item))
ings5 = new_list

In [133]:
ings5

[['sweet Italian sausage',
  'lean ground beef',
  'minced onion',
  'garlic, crushed',
  '(28 ounce) can crushed tomatoes',
  '(6 ounce) cans tomato paste',
  '(6.5 ounce) cans canned tomato sauce',
  'water',
  'white sugar',
  'dried basil leaves',
  'fennel seeds',
  'Italian seasoning',
  'salt, divided, or to taste',
  'ground black pepper',
  'chopped fresh parsley',
  'lasagna noodles',
  'ricotta cheese',
  'egg',
  'mozzarella cheese, sliced',
  'grated Parmesan cheese'],
 ['boneless pork chops',
  'paprika',
  'kosher salt and ground black pepper to taste',
  'butter, divided',
  '(8 ounce) package sliced fresh mushrooms',
  'garlic, minced',
  'Dijon mustard',
  'all-purpose flour',
  'beef broth'],
 ['cubed beef stew meat',
  'salt and ground black pepper to taste',
  'onion, chopped',
  '(10.75 ounce) can condensed cream of mushroom soup',
  'water',
  'dried chives',
  'garlic, minced',
  'Worcestershire sauce',
  'cube beef bouillon',
  'red wine',
  'cornstarch',
  'al

In [160]:
removing = 'crushed', 'divided', 'or to taste', 'sliced', 'minced', 'softened', 'zested', 'zested and juiced', 'cored and chopped', 
'cored and cut into 1 1/2-inch pieces', 'skewered', 'juiced',
'or more as needed', 'or more if needed', 'skinless', 'foil',
'an ', 'y color - cut in half lengthwise','cored', 'an ', 'd seeded',
'<sup>&reg;</sup>eynolds Wrap(<sup>&reg;</sup>) Heavy Duty Aluminum Foil',
'quartered', 'cut into 1/2-inch cubes', 'cut into 1/4-inch thick rings',
'skin removed', 'or as needed', 'or more to taste', 'thinly sliced', '-inch thick',
'cooked', 'drained', 'beaten', '(<sup>&reg;</sup>', 'cut into thin strips', 
'softened', 'pitted and diced', 'peeled', 'halved', 'pitted', 'cut in 1/2-inch cubes',
'cored', 'plus more to taste', 'drained and rinsed', 'crushed', 'Topping:', 'Pork Chops:',
'salt and ground black pepper to taste', 'rinsed and drained', 'split', 'heated',
'cored and separated into florets', 'peeled and sliced', 'halved crosswise and split',
'Marinade:', 'stems removed', 'leaves stripped', 'zested', 'For roasting:',
'or as needed to cover', 'skinless', 'seeded and cubed', 'room temperature', 'a t',
'for brushing', 'Your favorite marinara-style pasta sauce', 'thinly sliced', 'thickly sliced',
'sliced into thin rings', 'for garnish', 'flat', 'patted dry', 'coarsely chopped', 'with skin',
'chopped', 'cut into 4 pieces', 'cut into 3 inch lengths', 'sliced into 4 inch pieces',
'thawed', 'beaten', 'julienned', 'cut into thin slices', '(optional)', '(Optional)',
'cut into bite-sized pieces', 'grated', 'boned', 'torn into small pieces', 'cut into pieces',
'halved lengthwise','salt and ground black pepper to taste', ', crushed'



('halved lengthwise', 'salt and ground black pepper to taste', ', crushed')

In [155]:
ings5[0]

['sweet Italian sausage',
 'lean ground beef',
 'minced onion',
 'garlic, crushed',
 '(28 ounce) can crushed tomatoes',
 '(6 ounce) cans tomato paste',
 '(6.5 ounce) cans canned tomato sauce',
 'water',
 'white sugar',
 'dried basil leaves',
 'fennel seeds',
 'Italian seasoning',
 'salt, divided, or to taste',
 'ground black pepper',
 'chopped fresh parsley',
 'lasagna noodles',
 'ricotta cheese',
 'egg',
 'mozzarella cheese, sliced',
 'grated Parmesan cheese']

In [136]:
# Create a dictionary to make a DataFrame from
alls = {'name':names,'ingredients':ings5}
df = pd.DataFrame(alls)
df.head()

,name,ingredients
0,World's Best Lasagna,"[sweet Italian sausage, lean ground beef, minced onion, garlic, crushed, (28 ounce) can crushed tomatoes, (6 ounce) cans tomato paste, (6.5 ounce) cans canned tomato sauce, water, white sugar, dried basil leaves, fennel seeds, Italian seasoning, salt, divided, or to taste, ground black pepper, chopped fresh parsley, lasagna noodles, ricotta cheese, egg, mozzarella cheese, sliced, grated Parmesan cheese]"
1,Pork Chops in Garlic Mushroom Sauce,"[boneless pork chops, paprika, kosher salt and ground black pepper to taste, butter, divided, (8 ounce) package sliced fresh mushrooms, garlic, minced, Dijon mustard, all-purpose flour, beef broth]"
2,Super Duper Slow Cooker Beef Stroganoff,"[cubed beef stew meat, salt and ground black pepper to taste, onion, chopped, (10.75 ounce) can condensed cream of mushroom soup, water, dried chives, garlic, minced, Worcestershire sauce, cube beef bouillon, red wine, cornstarch, all-purpose flour, (8 ounce) container sour cream, (8 ounce) package sliced fresh mushrooms, cream cheese, chopped fresh parsley]"
3,Chef John's Perfect Prime Rib,"[prime rib roast, unsalted butter, softened, freshly ground black pepper, herbes de Provence, kosher salt]"
4,Beef Stroganoff for Instant Pot®,"[canola oil, onion, diced, salt, divided, beef stew meat, cut into 1-inch cubes, freshly ground black pepper, garlic, minced, dried thyme, soy sauce, chopped mushrooms, all-purpose flour, chicken broth, (16 ounce) package wide egg noodles, sour cream, or to taste]"


In [94]:
ings3

[[('1 pound', 'sweet Italian sausage'),
  ('3/4 pound', 'lean ground beef'),
  ('1/2 cup', 'minced onion'),
  ('2 cloves', 'garlic, crushed'),
  ('1 ', '(28 ounce) can crushed tomatoes'),
  ('2 ', '(6 ounce) cans tomato paste'),
  ('2 ', '(6.5 ounce) cans canned tomato sauce'),
  ('1/2 cup', 'water'),
  ('2 tablespoons', 'white sugar'),
  ('1 1/2 teaspoons', 'dried basil leaves'),
  ('1/2 teaspoon', 'fennel seeds'),
  ('1 teaspoon', 'Italian seasoning'),
  ('1 1/2 teaspoons', 'salt, divided, or to taste'),
  ('1/4 teaspoon', 'ground black pepper'),
  ('4 tablespoons', 'chopped fresh parsley'),
  ('12 ', 'lasagna noodles'),
  ('16 ounces', 'ricotta cheese'),
  ('1 ', 'egg'),
  ('3/4 pound', 'mozzarella cheese, sliced'),
  ('3/4 cup', 'grated Parmesan cheese')],
 [('2 pounds', 'boneless pork chops'),
  ('1/2 teaspoon', 'paprika'),
  ('1 pinch', 'kosher salt and ground black pepper to taste'),
  ('1/4 cup', 'butter, divided'),
  ('1 ', '(8 ounce) package sliced fresh mushrooms'),
  ('4 cl

In [54]:
len(ingy)

1024

In [44]:
type(ings1)

tuple

In [18]:
ings[1]

['2 pounds boneless pork chops',
 '1/2 teaspoon paprika',
 '1 pinch kosher salt and ground black pepper to taste',
 '1/4 cup butter, divided',
 '1 (8 ounce) package sliced fresh mushrooms',
 '4 cloves garlic, minced',
 '1 teaspoon Dijon mustard',
 '2 tablespoons all-purpose flour',
 '2 cups beef broth']

In [ ]:
remove = ['[ ', ']']

for val in df.ingredients:
#     val = val.strip("  ''")
    val = val.replace('[','').replace(']','').replace("'",'')
    print(parse(val))

In [ ]:
for row in df.ingredients:
    for val in row.split(','):
        print(parse(val.replace('[','').replace(']','').replace("'",'')))

In [6]:
# Create a dictionary to make a DataFrame from
alls = {'name':names,'ingredients':ings}
df = pd.DataFrame(alls)

# Preview DataFrame
df.head()

,name,ingredients
0,World's Best Lasagna,"[1 pound sweet Italian sausage, 3/4 pound lean ground beef, 1/2 cup minced onion, 2 cloves garlic, crushed, 1 (28 ounce) can crushed tomatoes, 2 (6 ounce) cans tomato paste, 2 (6.5 ounce) cans canned tomato sauce, 1/2 cup water, 2 tablespoons white sugar, 1 1/2 teaspoons dried basil leaves, 1/2 teaspoon fennel seeds, 1 teaspoon Italian seasoning, 1 1/2 teaspoons salt, divided, or to taste, 1/4 teaspoon ground black pepper, 4 tablespoons chopped fresh parsley, 12 lasagna noodles, 16 ounces ricotta cheese, 1 egg, 3/4 pound mozzarella cheese, sliced, 3/4 cup grated Parmesan cheese]"
1,Pork Chops in Garlic Mushroom Sauce,"[2 pounds boneless pork chops, 1/2 teaspoon paprika, 1 pinch kosher salt and ground black pepper to taste, 1/4 cup butter, divided, 1 (8 ounce) package sliced fresh mushrooms, 4 cloves garlic, minced, 1 teaspoon Dijon mustard, 2 tablespoons all-purpose flour, 2 cups beef broth]"
2,Super Duper Slow Cooker Beef Stroganoff,"[1 1/2 pounds cubed beef stew meat, salt and ground black pepper to taste, 1 onion, chopped, 1 (10.75 ounce) can condensed cream of mushroom soup, 1/4 cup water, 1 tablespoon dried chives, 2 cloves garlic, minced, 1 tablespoon Worcestershire sauce, 1 cube beef bouillon, 1/2 cup red wine, 1 tablespoon cornstarch, 1 tablespoon all-purpose flour, 1 (8 ounce) container sour cream, 1 (8 ounce) package sliced fresh mushrooms, 4 ounces cream cheese, 1/2 cup chopped fresh parsley]"
3,Chef John's Perfect Prime Rib,"[4 pounds prime rib roast, 1/4 cup unsalted butter, softened, 1 tablespoon freshly ground black pepper, 1 teaspoon herbes de Provence, kosher salt]"
4,Beef Stroganoff for Instant Pot®,"[2 tablespoons canola oil, 1/2 onion, diced, 2 teaspoons salt, divided, 2 pounds beef stew meat, cut into 1-inch cubes, 1 teaspoon freshly ground black pepper, 3 cloves garlic, minced, 1/2 teaspoon dried thyme, 2 tablespoons soy sauce, 3 cups chopped mushrooms, 2 tablespoons all-purpose flour, 3 cups chicken broth, 1 (16 ounce) package wide egg noodles, 3/4 cup sour cream, or to taste]"


In [9]:
df.sample(20)

,name,ingredients
294,Stuffed Shells III,"[1 (12 ounce) package jumbo pasta shells, 2 eggs, beaten, 1 (32 ounce) container ricotta cheese, 1 pound shredded mozzarella cheese, divided, 8 ounces grated Parmesan cheese, divided, 1 tablespoon dried parsley, 2 teaspoons salt, 1 teaspoon ground black pepper, 1 (28 ounce) jar pasta sauce, 8 ounces sliced fresh mushrooms]"
49,"Bow Ties with Sausage, Tomatoes and Cream","[1 (12 ounce) package bow tie pasta, 2 tablespoons olive oil, 1 pound sweet Italian sausage, casings removed and crumbled, 1/2 teaspoon red pepper flakes, 1/2 cup diced onion, 3 cloves garlic, minced, 1 (28 ounce) can Italian-style plum tomatoes, drained and coarsely chopped, 1 1/2 cups heavy cream, 1/2 teaspoon salt, 3 tablespoons minced fresh parsley]"
468,Maria's Pepper Steak,"[2 tablespoons olive oil, 1 medium onion, chopped, 2 large bell peppers, sliced into thin strips, 2 cloves garlic, minced, 1/3 cup soy sauce, 1/3 cup honey, 1/3 cup red wine vinegar, 1 1/2 pounds flank steak, cut into thin strips]"
350,Sesame Beef,"[1 pound round steak, 4 tablespoons soy sauce, 4 tablespoons white sugar, 4 tablespoons vegetable oil, 2 cloves garlic, minced, 2 green onions, chopped, 2 tablespoons sesame seeds]"
380,Perfect Sesame Chicken,"[2 tablespoons all-purpose flour, 2 tablespoons cornstarch, ¼ teaspoon baking soda, ¼ teaspoon baking powder, 2 tablespoons low-sodium soy sauce, 1 tablespoon dry sherry, 2 tablespoons water, 1 teaspoon vegetable oil, 1 dash sesame oil, 1 pound skinless, boneless chicken breast meat - cubed, 1 cup chicken broth, 1 cup white sugar, 2 tablespoons distilled white vinegar, 2 tablespoons dark soy sauce, 2 tablespoons sesame oil, 1 teaspoon chile paste, 1 clove garlic, minced, ¼ cup cornstarch, ½ cup water, 1 quart olive oil for frying, 2 tablespoons toasted sesame seeds]"
400,Lemon Cream Pasta with Chicken,"[3 skinless, boneless chicken breast halves, 1 lemon, quartered, 2 teaspoons garlic powder, divided, 1 teaspoon ground black pepper, divided, 2 (14.5 ounce) cans chicken broth, ¼ cup fresh lemon juice, 1 (8 ounce) package rotelle pasta, 1 cup heavy cream, 1 teaspoon grated lemon zest]"
478,Potato Salmon Patties,"[1 (14.75 ounce) can salmon, drained and flaked, 2 eggs, beaten, 1/4 cup garlic and herb seasoned dry bread crumbs, 1/4 cup dry potato flakes, 1 medium onion, minced, 1 clove garlic, minced, 1/4 teaspoon dried dill weed, 1/4 teaspoon celery salt, salt and pepper to taste, 2 tablespoons olive oil]"
192,Key West Chicken,"[3 tablespoons soy sauce, 1 tablespoon honey, 1 tablespoon vegetable oil, 1 teaspoon lime juice, 1 teaspoon chopped garlic, 4 skinless, boneless chicken breast halves]"
124,Spaghetti Squash I,"[1 spaghetti squash, halved lengthwise and seeded, 2 tablespoons vegetable oil, 1 onion, chopped, 1 clove garlic, minced, 1 1/2 cups chopped tomatoes, 3/4 cup crumbled feta cheese, 3 tablespoons sliced black olives, 2 tablespoons chopped fresh basil]"
184,Greek Chicken Pasta,"[1 (16 ounce) package linguine pasta, ½ cup chopped red onion, 1 tablespoon olive oil, 2 cloves garlic, crushed, 1 pound skinless, boneless chicken breast meat - cut into bite-size pieces, 1 (14 ounce) can marinated artichoke hearts, drained and chopped, 1 large tomato, chopped, ½ cup crumbled feta cheese, 3 tablespoons chopped fresh parsley, 2 tablespoons lemon juice, 2 teaspoons dried oregano, salt and pepper to taste, 2 lemons, wedged, for garnish]"


In [7]:
len(df)

591

In [20]:
from tqdm import tqdm_notebook as tqdm

In [23]:
# get ratings for recipes

ratings = []

for recipe in tqdm(recipes):
    responses = requests.get(recipe)
    soup = BeautifulSoup(responses.content,'html.parser')
    rating = soup.findAll('div',{'class':'rating-stars'})
    for r in rating:
        ratings.append(r.text)

In [108]:
ratings = []

responses = requests.get(recipes[2])
soup = BeautifulSoup(responses.content, 'html.parser')
rating = soup.find('meta', property='og:rating')
print(rating['content'] if rating else 'No rating')
# for r in rating:
#     ratings.append(r.text)

4.58854150772095


In [175]:
ratings = []

for recipe in tqdm(recipes):
    responses = requests.get(recipe)
    soup = BeautifulSoup(responses.content, 'html.parser')
    rating = soup.find('meta', property='og:rating')
    rating1 = soup.find('span', {'class':'review-star-text'}) 
    ratings.append(rating['content'] if rating else rating1.text)

In [203]:
upd_rate = []
for rate in ratings:
    if len(rate) > 30:
        upd_rate.append(rate[39:43])
    else:
        upd_rate.append(rate)

In [205]:
len(upd_rate)

591

In [227]:
for rate in df.ratings:
    round(rate, 3)

In [228]:
df.head()

,name,ingredients,url,ratings
0,World's Best Lasagna,"[1 pound sweet Italian sausage, 3/4 pound lean ground beef, 1/2 cup minced onion, 2 cloves garlic, crushed, 1 (28 ounce) can crushed tomatoes, 2 (6 ounce) cans tomato paste, 2 (6.5 ounce) cans canned tomato sauce, 1/2 cup water, 2 tablespoons white sugar, 1 1/2 teaspoons dried basil leaves, 1/2 teaspoon fennel seeds, 1 teaspoon Italian seasoning, 1 1/2 teaspoons salt, divided, or to taste, 1/4 teaspoon ground black pepper, 4 tablespoons chopped fresh parsley, 12 lasagna noodles, 16 ounces ricotta cheese, 1 egg, 3/4 pound mozzarella cheese, sliced, 3/4 cup grated Parmesan cheese]",https://www.allrecipes.com/recipe/23600/worlds-best-lasagna/,4.795724
1,Pork Chops in Garlic Mushroom Sauce,"[2 pounds boneless pork chops, 1/2 teaspoon paprika, 1 pinch kosher salt and ground black pepper to taste, 1/4 cup butter, divided, 1 (8 ounce) package sliced fresh mushrooms, 4 cloves garlic, minced, 1 teaspoon Dijon mustard, 2 tablespoons all-purpose flour, 2 cups beef broth]",https://www.allrecipes.com/recipe/269592/pork-chops-in-garlic-mushroom-sauce/,4.572222
2,Super Duper Slow Cooker Beef Stroganoff,"[1 1/2 pounds cubed beef stew meat, salt and ground black pepper to taste, 1 onion, chopped, 1 (10.75 ounce) can condensed cream of mushroom soup, 1/4 cup water, 1 tablespoon dried chives, 2 cloves garlic, minced, 1 tablespoon Worcestershire sauce, 1 cube beef bouillon, 1/2 cup red wine, 1 tablespoon cornstarch, 1 tablespoon all-purpose flour, 1 (8 ounce) container sour cream, 1 (8 ounce) package sliced fresh mushrooms, 4 ounces cream cheese, 1/2 cup chopped fresh parsley]",https://www.allrecipes.com/recipe/236867/super-duper-slow-cooker-beef-stroganoff/,4.588542
3,Chef John's Perfect Prime Rib,"[4 pounds prime rib roast, 1/4 cup unsalted butter, softened, 1 tablespoon freshly ground black pepper, 1 teaspoon herbes de Provence, kosher salt]",https://www.allrecipes.com/recipe/221958/chef-johns-perfect-prime-rib/,4.771789
4,Beef Stroganoff for Instant Pot®,"[2 tablespoons canola oil, 1/2 onion, diced, 2 teaspoons salt, divided, 2 pounds beef stew meat, cut into 1-inch cubes, 1 teaspoon freshly ground black pepper, 3 cloves garlic, minced, 1/2 teaspoon dried thyme, 2 tablespoons soy sauce, 3 cups chopped mushrooms, 2 tablespoons all-purpose flour, 3 cups chicken broth, 1 (16 ounce) package wide egg noodles, 3/4 cup sour cream, or to taste]",https://www.allrecipes.com/recipe/258468/beef-stroganoff-for-instant-pot/,4.457272


In [207]:
ratings1 ={'ratings':upd_rate}
df['ratings'] = pd.DataFrame(ratings1)
df.head()

,name,ingredients,rating,url,ratings
0,World's Best Lasagna,"[1 pound sweet Italian sausage, 3/4 pound lean ground beef, 1/2 cup minced onion, 2 cloves garlic, crushed, 1 (28 ounce) can crushed tomatoes, 2 (6 ounce) cans tomato paste, 2 (6.5 ounce) cans canned tomato sauce, 1/2 cup water, 2 tablespoons white sugar, 1 1/2 teaspoons dried basil leaves, 1/2 teaspoon fennel seeds, 1 teaspoon Italian seasoning, 1 1/2 teaspoons salt, divided, or to taste, 1/4 teaspoon ground black pepper, 4 tablespoons chopped fresh parsley, 12 lasagna noodles, 16 ounces ricotta cheese, 1 egg, 3/4 pound mozzarella cheese, sliced, 3/4 cup grated Parmesan cheese]",4.7957239151001,https://www.allrecipes.com/recipe/23600/worlds-best-lasagna/,4.7957239151001
1,Pork Chops in Garlic Mushroom Sauce,"[2 pounds boneless pork chops, 1/2 teaspoon paprika, 1 pinch kosher salt and ground black pepper to taste, 1/4 cup butter, divided, 1 (8 ounce) package sliced fresh mushrooms, 4 cloves garlic, minced, 1 teaspoon Dijon mustard, 2 tablespoons all-purpose flour, 2 cups beef broth]",4.5722222328186,https://www.allrecipes.com/recipe/269592/pork-chops-in-garlic-mushroom-sauce/,4.5722222328186
2,Super Duper Slow Cooker Beef Stroganoff,"[1 1/2 pounds cubed beef stew meat, salt and ground black pepper to taste, 1 onion, chopped, 1 (10.75 ounce) can condensed cream of mushroom soup, 1/4 cup water, 1 tablespoon dried chives, 2 cloves garlic, minced, 1 tablespoon Worcestershire sauce, 1 cube beef bouillon, 1/2 cup red wine, 1 tablespoon cornstarch, 1 tablespoon all-purpose flour, 1 (8 ounce) container sour cream, 1 (8 ounce) package sliced fresh mushrooms, 4 ounces cream cheese, 1/2 cup chopped fresh parsley]",4.58854150772095,https://www.allrecipes.com/recipe/236867/super-duper-slow-cooker-beef-stroganoff/,4.58854150772095
3,Chef John's Perfect Prime Rib,"[4 pounds prime rib roast, 1/4 cup unsalted butter, softened, 1 tablespoon freshly ground black pepper, 1 teaspoon herbes de Provence, kosher salt]",4.77178907394409,https://www.allrecipes.com/recipe/221958/chef-johns-perfect-prime-rib/,4.77178907394409
4,Beef Stroganoff for Instant Pot®,"[2 tablespoons canola oil, 1/2 onion, diced, 2 teaspoons salt, divided, 2 pounds beef stew meat, cut into 1-inch cubes, 1 teaspoon freshly ground black pepper, 3 cloves garlic, minced, 1/2 teaspoon dried thyme, 2 tablespoons soy sauce, 3 cups chopped mushrooms, 2 tablespoons all-purpose flour, 3 cups chicken broth, 1 (16 ounce) package wide egg noodles, 3/4 cup sour cream, or to taste]",4.45727157592773,https://www.allrecipes.com/recipe/258468/beef-stroganoff-for-instant-pot/,4.45727157592773


In [213]:
df.drop(columns='rating',inplace=True)

In [124]:
rateinfo={'rating':ratings}
df['rating']=pd.DataFrame(rateinfo)

In [126]:
df.drop(columns=['ratings'],inplace=True)

In [229]:
df.sort_values(by=['ratings'])

,name,ingredients,url,ratings
15,Gemelli Pasta with Roasted Pumpkin and Pancetta,"[2 cups peeled, cubed pumpkin, 2 tablespoons extra-virgin olive oil, divided, 1 teaspoon fresh thyme, 1/4 teaspoon salt, 1/8 teaspoon ground black pepper, 1/2 (16 ounce) package gemelli pasta, 1 (4 ounce) package diced pancetta, 1 cup chicken broth, 1/4 cup grated Parmigiano-Reggiano cheese]",https://www.allrecipes.com/recipe/276904/gemelli-pasta-with-roasted-pumpkin-and-pancetta/,0.000000
13,Al's Burmese Chicken Curry,"[1 teaspoon butter, or as needed, 8 shallots, thinly sliced, 3 tablespoons red curry paste, or more to taste, 2 tablespoons hot curry powder, 1 tablespoon ground red chile pepper, 2 1/2 pounds chicken thighs, 1 lemongrass, smashed and cut into 1-inch pieces, 10 bird's eye chile peppers, chopped, or to taste, 1 tablespoon ground coriander, 2 kaffir lime leaves, or to taste, 3 tablespoons fish sauce, water to cover, 1 (14 ounce) can coconut milk, 4 tomatoes, quartered, 1/2 small bunch fresh cilantro, chopped, or to taste]",https://www.allrecipes.com/recipe/276989/als-burmese-chicken-curry/,0.000000
82,Honey-Garlic Slow Cooker Chicken Thighs,"[4 skinless, boneless chicken thighs, ½ cup soy sauce, ½ cup ketchup, ⅓ cup honey, 3 cloves garlic, minced, 1 teaspoon dried basil]",https://www.allrecipes.com/recipe/236609/honey-garlic-slow-cooker-chicken-thighs/,3.940000
501,Pad Thai,"[1 (12 ounce) package rice noodles, 2 tablespoons butter, 1 pound boneless, skinless chicken breast halves, cut into bite-sized pieces, ¼ cup vegetable oil, 4 eggs, 1 tablespoon white wine vinegar, 2 tablespoons fish sauce, 3 tablespoons white sugar, ⅛ tablespoon crushed red pepper, 2 cups bean sprouts, ¼ cup crushed peanuts, 3 green onions, chopped, 1 lemon, cut into wedges]",https://www.allrecipes.com/recipe/42968/pad-thai/,3.950000
223,Pork Chops for the Slow Cooker,"[6 boneless pork chops, 1/4 cup brown sugar, 1 teaspoon ground ginger, 1/2 cup soy sauce, 1/4 cup ketchup, 2 cloves garlic, crushed, salt and pepper to taste]",https://www.allrecipes.com/recipe/25150/pork-chops-for-the-slow-cooker/,3.964794
474,Kathy's Delicious Whole Slow Cooker Chicken,"[1 (3 pound) whole chicken, skin removed, ½ cup chicken broth, ⅓ cup soy sauce, ⅓ cup olive oil, ¼ cup honey, 1 teaspoon Worcestershire sauce, 2 teaspoons balsamic vinegar, 2 teaspoons lemon juice, 1 teaspoon sesame oil, 2 tablespoons minced garlic]",https://www.allrecipes.com/recipe/72133/kathys-delicious-whole-slow-cooker-chicken/,3.990000
433,Slow Cooker Adobo Chicken,"[1 small sweet onion, sliced, 8 cloves garlic, crushed, ¾ cup low sodium soy sauce, ½ cup vinegar, 1 (3 pound) whole chicken, cut into pieces]",https://www.allrecipes.com/recipe/15650/slow-cooker-adobo-chicken/,3.990000
16,Beef Stroganoff with Ground Beef,"[2 tablespoons butter, 1 cup sliced fresh mushrooms, 1/3 cup chopped onion, 1 pound lean ground beef, 2 tablespoons all-purpose flour, 1 teaspoon garlic powder, 1 teaspoon seasoning salt, 1/2 teaspoon ground paprika, 1/4 teaspoon ground black pepper, 1 (10.75 ounce) can condensed cream of mushroom soup, 3/4 cup sour cream, or more to taste, 1 tablespoon liquid smoke, or to taste (optional), 1 tablespoon Worcestershire sauce]",https://www.allrecipes.com/recipe/276859/beef-stroganoff-with-ground-beef/,4.000000
398,Slow Cooker Chicken Cacciatore,"[6 skinless, boneless chicken breast halves, 1 (28 ounce) jar spaghetti sauce, 2 green bell pepper, seeded and cubed, 8 ounces fresh mushrooms, sliced, 1 onion, finely diced, 2 tablespoons minced garlic]",https://www.allrecipes.com/recipe/8817/slow-cooker-chicken-cacciatore/,4.010000
453,Simple Beef Stroganoff,"[1 (8 ounce) package egg noodles, 1 pound ground beef, 1 (10.75 ounce) can fat free condensed cream of mushroom soup, 1 tablespoon garlic powder, 1/2 cup sour cream, salt and pepper to taste]",https://www.allrecipes.com/recipe/16311/simple-beef-stroganoff/,4.062422


In [222]:
df.ratings = df.ratings.astype(float)

In [257]:
df2 = df

In [258]:
df2.head()

,name,ingredients,url,ratings
0,World's Best Lasagna,"[1 pound sweet Italian sausage, 3/4 pound lean ground beef, 1/2 cup minced onion, 2 cloves garlic, crushed, 1 (28 ounce) can crushed tomatoes, 2 (6 ounce) cans tomato paste, 2 (6.5 ounce) cans canned tomato sauce, 1/2 cup water, 2 tablespoons white sugar, 1 1/2 teaspoons dried basil leaves, 1/2 teaspoon fennel seeds, 1 teaspoon Italian seasoning, 1 1/2 teaspoons salt, divided, or to taste, 1/4 teaspoon ground black pepper, 4 tablespoons chopped fresh parsley, 12 lasagna noodles, 16 ounces ricotta cheese, 1 egg, 3/4 pound mozzarella cheese, sliced, 3/4 cup grated Parmesan cheese]",https://www.allrecipes.com/recipe/23600/worlds-best-lasagna/,4.795724
1,Pork Chops in Garlic Mushroom Sauce,"[2 pounds boneless pork chops, 1/2 teaspoon paprika, 1 pinch kosher salt and ground black pepper to taste, 1/4 cup butter, divided, 1 (8 ounce) package sliced fresh mushrooms, 4 cloves garlic, minced, 1 teaspoon Dijon mustard, 2 tablespoons all-purpose flour, 2 cups beef broth]",https://www.allrecipes.com/recipe/269592/pork-chops-in-garlic-mushroom-sauce/,4.572222
2,Super Duper Slow Cooker Beef Stroganoff,"[1 1/2 pounds cubed beef stew meat, salt and ground black pepper to taste, 1 onion, chopped, 1 (10.75 ounce) can condensed cream of mushroom soup, 1/4 cup water, 1 tablespoon dried chives, 2 cloves garlic, minced, 1 tablespoon Worcestershire sauce, 1 cube beef bouillon, 1/2 cup red wine, 1 tablespoon cornstarch, 1 tablespoon all-purpose flour, 1 (8 ounce) container sour cream, 1 (8 ounce) package sliced fresh mushrooms, 4 ounces cream cheese, 1/2 cup chopped fresh parsley]",https://www.allrecipes.com/recipe/236867/super-duper-slow-cooker-beef-stroganoff/,4.588542
3,Chef John's Perfect Prime Rib,"[4 pounds prime rib roast, 1/4 cup unsalted butter, softened, 1 tablespoon freshly ground black pepper, 1 teaspoon herbes de Provence, kosher salt]",https://www.allrecipes.com/recipe/221958/chef-johns-perfect-prime-rib/,4.771789
4,Beef Stroganoff for Instant Pot®,"[2 tablespoons canola oil, 1/2 onion, diced, 2 teaspoons salt, divided, 2 pounds beef stew meat, cut into 1-inch cubes, 1 teaspoon freshly ground black pepper, 3 cloves garlic, minced, 1/2 teaspoon dried thyme, 2 tablespoons soy sauce, 3 cups chopped mushrooms, 2 tablespoons all-purpose flour, 3 cups chicken broth, 1 (16 ounce) package wide egg noodles, 3/4 cup sour cream, or to taste]",https://www.allrecipes.com/recipe/258468/beef-stroganoff-for-instant-pot/,4.457272


In [261]:
df2['ingredients'] = df2['ingredients'].apply(', '.join)

In [262]:
df2.head()

,name,ingredients,url,ratings
0,World's Best Lasagna,"1 pound sweet Italian sausage, 3/4 pound lean ground beef, 1/2 cup minced onion, 2 cloves garlic, crushed, 1 (28 ounce) can crushed tomatoes, 2 (6 ounce) cans tomato paste, 2 (6.5 ounce) cans canned tomato sauce, 1/2 cup water, 2 tablespoons white sugar, 1 1/2 teaspoons dried basil leaves, 1/2 teaspoon fennel seeds, 1 teaspoon Italian seasoning, 1 1/2 teaspoons salt, divided, or to taste, 1/4 teaspoon ground black pepper, 4 tablespoons chopped fresh parsley, 12 lasagna noodles, 16 ounces ricotta cheese, 1 egg, 3/4 pound mozzarella cheese, sliced, 3/4 cup grated Parmesan cheese",https://www.allrecipes.com/recipe/23600/worlds-best-lasagna/,4.795724
1,Pork Chops in Garlic Mushroom Sauce,"2 pounds boneless pork chops, 1/2 teaspoon paprika, 1 pinch kosher salt and ground black pepper to taste, 1/4 cup butter, divided, 1 (8 ounce) package sliced fresh mushrooms, 4 cloves garlic, minced, 1 teaspoon Dijon mustard, 2 tablespoons all-purpose flour, 2 cups beef broth",https://www.allrecipes.com/recipe/269592/pork-chops-in-garlic-mushroom-sauce/,4.572222
2,Super Duper Slow Cooker Beef Stroganoff,"1 1/2 pounds cubed beef stew meat, salt and ground black pepper to taste, 1 onion, chopped, 1 (10.75 ounce) can condensed cream of mushroom soup, 1/4 cup water, 1 tablespoon dried chives, 2 cloves garlic, minced, 1 tablespoon Worcestershire sauce, 1 cube beef bouillon, 1/2 cup red wine, 1 tablespoon cornstarch, 1 tablespoon all-purpose flour, 1 (8 ounce) container sour cream, 1 (8 ounce) package sliced fresh mushrooms, 4 ounces cream cheese, 1/2 cup chopped fresh parsley",https://www.allrecipes.com/recipe/236867/super-duper-slow-cooker-beef-stroganoff/,4.588542
3,Chef John's Perfect Prime Rib,"4 pounds prime rib roast, 1/4 cup unsalted butter, softened, 1 tablespoon freshly ground black pepper, 1 teaspoon herbes de Provence, kosher salt",https://www.allrecipes.com/recipe/221958/chef-johns-perfect-prime-rib/,4.771789
4,Beef Stroganoff for Instant Pot®,"2 tablespoons canola oil, 1/2 onion, diced, 2 teaspoons salt, divided, 2 pounds beef stew meat, cut into 1-inch cubes, 1 teaspoon freshly ground black pepper, 3 cloves garlic, minced, 1/2 teaspoon dried thyme, 2 tablespoons soy sauce, 3 cups chopped mushrooms, 2 tablespoons all-purpose flour, 3 cups chicken broth, 1 (16 ounce) package wide egg noodles, 3/4 cup sour cream, or to taste",https://www.allrecipes.com/recipe/258468/beef-stroganoff-for-instant-pot/,4.457272


In [231]:
ings[1]

['2 pounds boneless pork chops',
 '1/2 teaspoon paprika',
 '1 pinch kosher salt and ground black pepper to taste',
 '1/4 cup butter, divided',
 '1 (8 ounce) package sliced fresh mushrooms',
 '4 cloves garlic, minced',
 '1 teaspoon Dijon mustard',
 '2 tablespoons all-purpose flour',
 '2 cups beef broth']

## NLP

In [232]:
import numpy as np
import spacy

In [233]:
from spacy.lang.en import English

In [249]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [234]:
nlp = English()

In [248]:
ings

[['1 pound sweet Italian sausage',
  '3/4 pound lean ground beef',
  '1/2 cup minced onion',
  '2 cloves garlic, crushed',
  '1 (28 ounce) can crushed tomatoes',
  '2 (6 ounce) cans tomato paste',
  '2 (6.5 ounce) cans canned tomato sauce',
  '1/2 cup water',
  '2 tablespoons white sugar',
  '1 1/2 teaspoons dried basil leaves',
  '1/2 teaspoon fennel seeds',
  '1 teaspoon Italian seasoning',
  '1 1/2 teaspoons salt, divided, or to taste',
  '1/4 teaspoon ground black pepper',
  '4 tablespoons chopped fresh parsley',
  '12 lasagna noodles',
  '16 ounces ricotta cheese',
  '1 egg',
  '3/4 pound mozzarella cheese, sliced',
  '3/4 cup grated Parmesan cheese'],
 ['2 pounds boneless pork chops',
  '1/2 teaspoon paprika',
  '1 pinch kosher salt and ground black pepper to taste',
  '1/4 cup butter, divided',
  '1 (8 ounce) package sliced fresh mushrooms',
  '4 cloves garlic, minced',
  '1 teaspoon Dijon mustard',
  '2 tablespoons all-purpose flour',
  '2 cups beef broth'],
 ['1 1/2 pounds cub

In [255]:
import nltk

In [263]:
df2['tokenized_sents'] = df2.apply(lambda row: nltk.word_tokenize(row['ingredients']), axis=1)

In [271]:
df2['tokenized_sents'] = df2['tokenized_sents'].apply(', '.join)

In [272]:
df2.head()

,name,ingredients,url,ratings,tokenized_sents
0,World's Best Lasagna,"1 pound sweet Italian sausage, 3/4 pound lean ground beef, 1/2 cup minced onion, 2 cloves garlic, crushed, 1 (28 ounce) can crushed tomatoes, 2 (6 ounce) cans tomato paste, 2 (6.5 ounce) cans canned tomato sauce, 1/2 cup water, 2 tablespoons white sugar, 1 1/2 teaspoons dried basil leaves, 1/2 teaspoon fennel seeds, 1 teaspoon Italian seasoning, 1 1/2 teaspoons salt, divided, or to taste, 1/4 teaspoon ground black pepper, 4 tablespoons chopped fresh parsley, 12 lasagna noodles, 16 ounces ricotta cheese, 1 egg, 3/4 pound mozzarella cheese, sliced, 3/4 cup grated Parmesan cheese",https://www.allrecipes.com/recipe/23600/worlds-best-lasagna/,4.795724,"1, pound, sweet, Italian, sausage, ,, 3/4, pound, lean, ground, beef, ,, 1/2, cup, minced, onion, ,, 2, cloves, garlic, ,, crushed, ,, 1, (, 28, ounce, ), can, crushed, tomatoes, ,, 2, (, 6, ounce, ), cans, tomato, paste, ,, 2, (, 6.5, ounce, ), cans, canned, tomato, sauce, ,, 1/2, cup, water, ,, 2, tablespoons, white, sugar, ,, 1, 1/2, teaspoons, dried, basil, leaves, ,, 1/2, teaspoon, fennel, seeds, ,, 1, teaspoon, Italian, seasoning, ,, 1, 1/2, teaspoons, salt, ,, divided, ,, or, to, taste, ,, 1/4, teaspoon, ground, black, pepper, ,, 4, tablespoons, chopped, fresh, parsley, ,, 12, lasagna, noodles, ,, 16, ounces, ricotta, cheese, ,, 1, egg, ,, 3/4, pound, mozzarella, cheese, ,, sliced, ,, 3/4, cup, grated, Parmesan, cheese"
1,Pork Chops in Garlic Mushroom Sauce,"2 pounds boneless pork chops, 1/2 teaspoon paprika, 1 pinch kosher salt and ground black pepper to taste, 1/4 cup butter, divided, 1 (8 ounce) package sliced fresh mushrooms, 4 cloves garlic, minced, 1 teaspoon Dijon mustard, 2 tablespoons all-purpose flour, 2 cups beef broth",https://www.allrecipes.com/recipe/269592/pork-chops-in-garlic-mushroom-sauce/,4.572222,"2, pounds, boneless, pork, chops, ,, 1/2, teaspoon, paprika, ,, 1, pinch, kosher, salt, and, ground, black, pepper, to, taste, ,, 1/4, cup, butter, ,, divided, ,, 1, (, 8, ounce, ), package, sliced, fresh, mushrooms, ,, 4, cloves, garlic, ,, minced, ,, 1, teaspoon, Dijon, mustard, ,, 2, tablespoons, all-purpose, flour, ,, 2, cups, beef, broth"
2,Super Duper Slow Cooker Beef Stroganoff,"1 1/2 pounds cubed beef stew meat, salt and ground black pepper to taste, 1 onion, chopped, 1 (10.75 ounce) can condensed cream of mushroom soup, 1/4 cup water, 1 tablespoon dried chives, 2 cloves garlic, minced, 1 tablespoon Worcestershire sauce, 1 cube beef bouillon, 1/2 cup red wine, 1 tablespoon cornstarch, 1 tablespoon all-purpose flour, 1 (8 ounce) container sour cream, 1 (8 ounce) package sliced fresh mushrooms, 4 ounces cream cheese, 1/2 cup chopped fresh parsley",https://www.allrecipes.com/recipe/236867/super-duper-slow-cooker-beef-stroganoff/,4.588542,"1, 1/2, pounds, cubed, beef, stew, meat, ,, salt, and, ground, black, pepper, to, taste, ,, 1, onion, ,, chopped, ,, 1, (, 10.75, ounce, ), can, condensed, cream, of, mushroom, soup, ,, 1/4, cup, water, ,, 1, tablespoon, dried, chives, ,, 2, cloves, garlic, ,, minced, ,, 1, tablespoon, Worcestershire, sauce, ,, 1, cube, beef, bouillon, ,, 1/2, cup, red, wine, ,, 1, tablespoon, cornstarch, ,, 1, tablespoon, all-purpose, flour, ,, 1, (, 8, ounce, ), container, sour, cream, ,, 1, (, 8, ounce, ), package, sliced, fresh, mushrooms, ,, 4, ounces, cream, cheese, ,, 1/2, cup, chopped, fresh, parsley"
3,Chef John's Perfect Prime Rib,"4 pounds prime rib roast, 1/4 cup unsalted butter, softened, 1 tablespoon freshly ground black pepper, 1 teaspoon herbes de Provence, kosher salt",https://www.allrecipes.com/recipe/221958/chef-johns-perfect-prime-rib/,4.771789,"4, pounds, prime, rib, roast, ,, 1/4, cup, unsalted, butter, ,, softened, ,, 1, tablespoon, freshly, ground, black, pepper, ,, 1, teaspoon, herbes, de, Provence, ,, kosher, salt"
4,Beef Stroganoff for Instant Pot®,"2 tablespoons canola oil, 1/2 onion, diced, 2 teaspoons salt, divided, 2 pounds beef stew meat, cu

In [293]:
nlp = spacy.load('en')

In [297]:
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA

In [299]:
import string 
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Creating our tokenizer function
def spacy_tokenizer(text):

    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(text)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [304]:
stop_words

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'forme

In [302]:
stop_words.update((range(0,10)))

In [307]:
stop_words.update(('to','taste','pound','ounce','cup','teaspoon','cans','ground',
                   'ounces','sliced','pounds','minced','divided','cups','can',
                  'softened','packet','and','or','tablespoon','small','coarsely',
                  'shredded', 'cubes','chopped','such','as','mince','crush','crushed',
                  'divide','slice'))

In [300]:
df2['ingredients'][0]

'1 pound sweet Italian sausage, 3/4 pound lean ground beef, 1/2 cup minced onion, 2 cloves garlic, crushed, 1 (28 ounce) can crushed tomatoes, 2 (6 ounce) cans tomato paste, 2 (6.5 ounce) cans canned tomato sauce, 1/2 cup water, 2 tablespoons white sugar, 1 1/2 teaspoons dried basil leaves, 1/2 teaspoon fennel seeds, 1 teaspoon Italian seasoning, 1 1/2 teaspoons salt, divided, or to taste, 1/4 teaspoon ground black pepper, 4 tablespoons chopped fresh parsley, 12 lasagna noodles, 16 ounces ricotta cheese, 1 egg, 3/4 pound mozzarella cheese, sliced, 3/4 cup grated Parmesan cheese'

In [308]:
text = df2['ingredients'][0]
spacy_tokenizer(text)

['1',
 'sweet',
 'italian',
 'sausage',
 '3/4',
 'lean',
 'beef',
 '1/2',
 'onion',
 '2',
 'clove',
 'garlic',
 '1',
 '28',
 'tomato',
 '2',
 '6',
 'tomato',
 'paste',
 '2',
 '6.5',
 'tomato',
 'sauce',
 '1/2',
 'water',
 '2',
 'white',
 'sugar',
 '1',
 '1/2',
 'dry',
 'basil',
 'leave',
 '1/2',
 'fennel',
 'seed',
 '1',
 'italian',
 'seasoning',
 '1',
 '1/2',
 'salt',
 '1/4',
 'black',
 'pepper',
 '4',
 'chop',
 'fresh',
 'parsley',
 '12',
 'lasagna',
 'noodle',
 '16',
 'ricotta',
 'cheese',
 '1',
 'egg',
 '3/4',
 'mozzarella',
 'cheese',
 '3/4',
 'grate',
 'parmesan',
 'cheese']

In [314]:
df2['tokenized_sents'] = df2.apply(lambda row: spacy_tokenizer(row['ingredients']),axis=1)

In [315]:
df2.head()

,name,ingredients,url,ratings,tokenized_sents
0,World's Best Lasagna,"1 pound sweet Italian sausage, 3/4 pound lean ground beef, 1/2 cup minced onion, 2 cloves garlic, crushed, 1 (28 ounce) can crushed tomatoes, 2 (6 ounce) cans tomato paste, 2 (6.5 ounce) cans canned tomato sauce, 1/2 cup water, 2 tablespoons white sugar, 1 1/2 teaspoons dried basil leaves, 1/2 teaspoon fennel seeds, 1 teaspoon Italian seasoning, 1 1/2 teaspoons salt, divided, or to taste, 1/4 teaspoon ground black pepper, 4 tablespoons chopped fresh parsley, 12 lasagna noodles, 16 ounces ricotta cheese, 1 egg, 3/4 pound mozzarella cheese, sliced, 3/4 cup grated Parmesan cheese",https://www.allrecipes.com/recipe/23600/worlds-best-lasagna/,4.795724,"[1, sweet, italian, sausage, 3/4, lean, beef, 1/2, onion, 2, clove, garlic, 1, 28, tomato, 2, 6, tomato, paste, 2, 6.5, tomato, sauce, 1/2, water, 2, white, sugar, 1, 1/2, dry, basil, leave, 1/2, fennel, seed, 1, italian, seasoning, 1, 1/2, salt, 1/4, black, pepper, 4, chop, fresh, parsley, 12, lasagna, noodle, 16, ricotta, cheese, 1, egg, 3/4, mozzarella, cheese, 3/4, grate, parmesan, cheese]"
1,Pork Chops in Garlic Mushroom Sauce,"2 pounds boneless pork chops, 1/2 teaspoon paprika, 1 pinch kosher salt and ground black pepper to taste, 1/4 cup butter, divided, 1 (8 ounce) package sliced fresh mushrooms, 4 cloves garlic, minced, 1 teaspoon Dijon mustard, 2 tablespoons all-purpose flour, 2 cups beef broth",https://www.allrecipes.com/recipe/269592/pork-chops-in-garlic-mushroom-sauce/,4.572222,"[2, boneless, pork, chop, 1/2, paprika, 1, pinch, kosher, salt, black, pepper, 1/4, butter, 1, 8, package, fresh, mushroom, 4, clove, garlic, 1, dijon, mustard, 2, purpose, flour, 2, beef, broth]"
2,Super Duper Slow Cooker Beef Stroganoff,"1 1/2 pounds cubed beef stew meat, salt and ground black pepper to taste, 1 onion, chopped, 1 (10.75 ounce) can condensed cream of mushroom soup, 1/4 cup water, 1 tablespoon dried chives, 2 cloves garlic, minced, 1 tablespoon Worcestershire sauce, 1 cube beef bouillon, 1/2 cup red wine, 1 tablespoon cornstarch, 1 tablespoon all-purpose flour, 1 (8 ounce) container sour cream, 1 (8 ounce) package sliced fresh mushrooms, 4 ounces cream cheese, 1/2 cup chopped fresh parsley",https://www.allrecipes.com/recipe/236867/super-duper-slow-cooker-beef-stroganoff/,4.588542,"[1, 1/2, cube, beef, stew, meat, salt, black, pepper, 1, onion, chop, 1, 10.75, condense, cream, mushroom, soup, 1/4, water, 1, dry, chive, 2, clove, garlic, 1, worcestershire, sauce, 1, cube, beef, bouillon, 1/2, red, wine, 1, cornstarch, 1, purpose, flour, 1, 8, container, sour, cream, 1, 8, package, fresh, mushroom, 4, cream, cheese, 1/2, chop, fresh, parsley]"
3,Chef John's Perfect Prime Rib,"4 pounds prime rib roast, 1/4 cup unsalted butter, softened, 1 tablespoon freshly ground black pepper, 1 teaspoon herbes de Provence, kosher salt",https://www.allrecipes.com/recipe/221958/chef-johns-perfect-prime-rib/,4.771789,"[4, prime, rib, roast, 1/4, unsalte, butter, soften, 1, freshly, grind, black, pepper, 1, herbe, de, provence, kosher, salt]"
4,Beef Stroganoff for Instant Pot®,"2 tablespoons canola oil, 1/2 onion, diced, 2 teaspoons salt, divided, 2 pounds beef stew meat, cut into 1-inch cubes, 1 teaspoon freshly ground black pepper, 3 cloves garlic, minced, 1/2 teaspoon dried thyme, 2 tablespoons soy sauce, 3 cups chopped mushrooms, 2 tablespoons all-purpose flour, 3 cups chicken broth, 1 (16 ounce) package wide egg noodles, 3/4 cup sour cream, or to taste",https://www.allrecipes.com/recipe/258468/beef-stroganoff-for-instant-pot/,4.457272,"[2, canola, oil, 1/2, onion, dice, 2, salt, 2, beef, stew, meat, cut, 1-inch, cube, 1, freshly, grind, black, pepper, 3, clove, garlic, 1/2, dry, thyme, 2, soy, sauce, 3, chop, mushroom, 2, purpose, flour, 3, chicken, broth, 1, 16, package, wide, egg, noodle, 3/4, sour, cream]"


In [274]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [275]:
stop_words = set(stopwords.words('english')) 

In [279]:
stop_words.update((range(0,10)))

In [282]:
stop_words.update(('to','taste','pound','ounce','cup','teaspoon','cans','ground',
                   'ounces','sliced','pounds','minced','divided','cups','can',
                  'softened','packet','and','or','tablespoon','small','coarsely',
                  'shredded', 'cubes','chopped','such','as'))

In [285]:
stop_words

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'cans',
 'chopped',
 'coarsely',
 'couldn',
 "couldn't",
 'cubes',
 'cup',
 'cups',
 'd',
 'did',
 'didn',
 "didn't",
 'divided',
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'ground',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'minced',
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'n

In [266]:
tokens = []
for ing in df2['tokenized_sents']:
    for g in ing:
        tokens.append(nlp(g))

In [286]:
filtered_sent = []
for word in df2['tokenized_sents']:
    if word not in stop_words:
        filtered_sent.append(word)

In [291]:
cleaned = []
for sent in filtered_sent:
    if sent not in stop_words:
        cleaned.append(sent)

In [334]:
def _filter(token):
    if len(token) < 2:
        return False
#     if token.is_stop:
#         return False
#     if token.text[0].islower():
#         return False
    if token.is_digit:
        return False
    if token.like_num:
        return False
    return True

In [328]:
def _clean(text):
    text = text.replace("(", "")
    text = text.split("/")[0]
    return text

In [341]:
df2['tokenized_sents'] = [_clean(t.text) for i in str(df2['tokenized_sents']) for t in nlp(i) if _filter(t) and len(_clean(t.text)) >= 2]

ValueError: Length of values does not match length of index

In [ ]:
df2.head()

In [338]:
exclude = ['pound', 'cup', 'chopped', 'minced', 'teaspoon', 
           'ounce', 'can', 'diced', 'without', 'inch', 'cups', 
           'shredded', 'of', 'tablespoon', 'drained', 'and', 
           'cubed', 'melted', 'prepared', 'package']

In [322]:
ings[230]

['1 pound ground beef',
 '1/2 cup chopped onion',
 '1 clove garlic, minced',
 '1/2 teaspoon cumin',
 '1/4 teaspoon salt',
 '1/8 teaspoon pepper',
 '1 (4.5 ounce) can diced green chile peppers',
 '1 (16 ounce) can refried beans',
 '1 (15 ounce) can chili without beans',
 '1 (10.75 ounce) can condensed tomato soup',
 '1 (10 ounce) can enchilada sauce',
 '6 (12 inch) flour tortillas, warmed',
 '2 cups shredded lettuce',
 '1 cup chopped tomatoes',
 '2 cups shredded Mexican blend cheese',
 '1/2 cup chopped green onions']

### Notes
Scraping 400 pages yields 7087 recipes <br>
Using Scrape_Me on 7087 recipes takes 60+ minutes